### Webscraping

## Table1: medicine_list
`name` - Name of the medicine

`size_of_the_bottle` - Size of the medicine bottle or pack

`MRP_of_the_bottle`  - MRP of the bottle

`price_of_the_bottle`  - Selling price of the bottle

`1mg_url` - 1mg url where the medicine sold

In [30]:
import csv
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

# Set up the WebDriver
driver = webdriver.Chrome()

# Set the URL of the first page
homeopathy_url = 'https://www.1mg.com/categories/homeopathy-57?filter=true&pageNumber=1'
driver.get(url=homeopathy_url)

# Making soup
soup = BeautifulSoup(driver.page_source, "html.parser")

# Remove dropdown
close_popup_button = driver.find_element(By.CLASS_NAME, 'UpdateCityModal__cancel-btn___2jWwS')
close_popup_button.click()
time.sleep(2)


# Create a list to store all the data
medicine_list=[]

total_number_of_pages = 30

# Iterate through the pages
for page_number in range(1, total_number_of_pages + 1):
    homeopathy_url = f'https://www.1mg.com/categories/homeopathy-57?filter=true&pageNumber={page_number}'
    driver.get(url=homeopathy_url)

    # Wait for some time to ensure the page is fully loaded
    time.sleep(2)

    # Making soup
    soup = BeautifulSoup(driver.page_source, "html.parser")

    blocks = soup.find_all("div", {"class": "style__product-box___liepi"})

    # Extracting the Name, Size, MRP , Sale price, URL's
    for i in blocks:

        name_of_medicine_element = i.find('div', {'class': 'style__pro-title___2QwJy'})
        name_of_medicine = name_of_medicine_element.text.strip() if name_of_medicine_element else 'NA'

        size_text = i.find('div', {'class': 'style__pack-size___2JQG7'}).text
        # Define a regular expression to extract the volume information
        volume_pattern = re.compile(r'\b(\d+\s*(?:ml|gm|tablets))\b', re.IGNORECASE)
        # Find all matches in the text
        matches = volume_pattern.findall(size_text)
        # Check if matches are present and print volumes or "NA"
        if matches:
            for size in matches:
                size 
        else:
            size = "NA"

        MRP_element = i.find("span", {"class": "style__discount-price___25Bya"})
        MRP = MRP_element.text.strip() if MRP_element else 'NA'

        sale_price_element = i.find("div", {"class": "style__price-tag___cOxYc"})
        sale_price = sale_price_element.text.strip() if sale_price_element else 'NA'

        url_element = i.find('a', {'class': 'style__product-link___UB_67'})
        url = f'https://www.1mg.com{url_element.get("href")}'

        # Store data in the list
        all_medicines_data.append([name_of_medicine, size, MRP, sale_price, url])

        dict={
            'name':name_of_medicine,
            'size_of_the_bottle':size_text,
            'MRP_of_the_bottle':MRP,
            'price_of_the_bottle':sale_price,
            '1mg_url':url
        }
        medicine_list.append(dict)
        
# print(medicine_list)
df=pd.DataFrame(medicine_list)
df.to_csv('medicine_list.csv',index=False)
print(df.head())


# Close the WebDriver
driver.quit()



                                                name  \
0  Dr Willmar Schwabe India Berberis Aquifolium M...   
1                      Medisynth Jondila Forte Syrup   
2                 Medisynth Jondila Sugar Free Syrup   
3         SBL Arnica Montana Herbal Shampoo with TJC   
4                      Medisynth Jondila Forte Syrup   

                size_of_the_bottle MRP_of_the_bottle price_of_the_bottle  \
0  bottle of 30 ml Mother Tincture              ₹220                ₹165   
1           bottle of 120 ml Syrup              ₹120                ₹107   
2           bottle of 200 ml Syrup              ₹170                ₹166   
3         bottle of 200 ml Shampoo                NA             MRP₹170   
4           bottle of 450 ml Syrup              ₹320                ₹300   

                                             1mg_url  
0  https://www.1mg.com/otc/dr-willmar-schwabe-ind...  
1  https://www.1mg.com/otc/medisynth-jondila-fort...  
2  https://www.1mg.com/otc/medisynth-jond

## Table - 2 : `medicine_details`

Attributes in table - 2:

`name` - Name of the medicine

`brand_name`  - Brand name

`key_benefits` - Key Benefits area (Hair, eye, joint, skin)

`key_ingredients` - Ingredient of medicine

`rating` - user rating of the medicine

`number_of_rating`  - Number of people rated that product

In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import requests
import gspread
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from gspread_dataframe import set_with_dataframe
import requests
import selenium.webdriver as webdriver
import time
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

firefox_binary_path = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options = Options()
options.binary_location = firefox_binary_path
driver = webdriver.Firefox(options=options)

# url list
data = pd.read_csv('medicine_list.csv')
url_list = data['1mg_url']

# Create an empty DataFrame with the desired column names
columns = ['Company', 'brand_name', 'key ingredients', 'key benefits', 'rating', 'number of rating']
df = pd.DataFrame(columns=columns)

cnt = 0

for j in url_list:
    url = j
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        company_name = soup.find('h1', {'class': 'ProductTitle__product-title___3QMYH'}).text
        brand_name = soup.find('div', {'class': 'ProductTitle__marketer___7Wsj9'}).text

        key_ingredients_element = soup.find('div', {'class': 'KeyIng__container___1cHvi'})
        key_ingredients_items = key_ingredients_element.find_all('li') if key_ingredients_element else []
        key_ingredients = [item.text.strip() for item in key_ingredients_items] if key_ingredients_element else 'NA'

        key_benefits_element = soup.find('div', {'class': 'ProductHighlights__product-highlights___2jAF5'})
        key_benefits_items = key_benefits_element.find_all('li') if key_benefits_element else []
        key_benefits = [item.text.strip() for item in key_benefits_items] if key_benefits_items else 'NA'

        rating_star = soup.find('div', {'class': 'RatingDisplay__ratings-container___3oUuo'}).text
        rating = soup.find('span', {'class': 'RatingDisplay__ratings-header___ZNj5b'}).text

        row_dict = {
            'Company': company_name,
            'brand_name': brand_name,
            'key ingredients': key_ingredients,
            'key benefits': key_benefits,
            'rating': rating_star,
            'number of rating': rating
        }
        df = pd.concat([df, pd.DataFrame([row_dict])], ignore_index=True)
        df.to_csv('medicine_details.csv', index=False)
        cnt += 1
        print(cnt)
    except Exception as e:
        print(f"Error processing URL: {url}. Error: {e}")

# Close the browser after scraping all URLs
driver.quit()

# Save the final DataFrame to CSV
# df.to_csv('medicine_details.csv', index=False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


### ----------------------------------------------------------------------------------------------------------------------------------

### trying for key ingredients

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import requests
import gspread
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from gspread_dataframe import set_with_dataframe
import requests
import selenium.webdriver as webdriver
import time
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

firefox_binary_path = r"C:\Program Files\Mozilla Firefox\firefox.exe"

# Create an instance of the Firefox Options class
options = Options()

# Set the binary location in the options
options.binary_location = firefox_binary_path

driver = webdriver.Firefox(options=options)

# url list
# url_list=[
#     'https://www.1mg.com/otc/dr-willmar-schwabe-india-alpha-coff-cough-syrup-otc407060',
#     'https://www.1mg.com/otc/dr.-reckeweg-r9-cough-drop-otc326072',
#     'https://www.1mg.com/otc/bakson-s-homeopathy-kali-phosphoricum-biochemic-tablet-30x-otc695194'
# ]
# url list 1300 products
data=pd.read_csv('medicine_list.csv')
url_list=data['1mg_url']
# print(url_list)


cnt = 0
details_list = []

for j in url_list:
    url = j
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        company_name = soup.find('h1', {'class': 'ProductTitle__product-title___3QMYH'}).text
        brand_name = soup.find('div', {'class': 'ProductTitle__marketer___7Wsj9'}).text

        # key_ingredients_element = soup.find('div', {'xpath': '//*[@id="container"]/div/div/div[3]/div[1]/div[2]/div[2]/ul[1]'})
        # key_ingredients_items = key_ingredients_element.find_all('li') if key_ingredients_element else []
        # key_ingredients = [item.text.strip() for item in key_ingredients_items] if key_ingredients_element else 'NA'
        keyingredients = soup.find('strong',string='Key Ingredients:')
        ingredients = keyingredients.find_next('li').string.strip() if keyingredients else ''
        print(ingredients)
        
        key_benefits_element = soup.find('div', {'class': 'ProductHighlights__product-highlights___2jAF5'})
        key_benefits_items = key_benefits_element.find_all('li') if key_benefits_element else []
        key_benefits = [item.text.strip() for item in key_benefits_items] if key_benefits_items else 'NA'

        key_ingredients = None
        for strong_tag in soup.find_all('strong'):
            if strong_tag.get_text() == 'Key Ingredients:':
                key_ingredients = strong_tag.find_next_sibling(text=True).strip()
                break

        print("Key Ingredients:", key_ingredients)
        rating_star = soup.find('div', {'class': 'RatingDisplay__ratings-container___3oUuo'}).text
        rating = soup.find('span', {'class': 'RatingDisplay__ratings-header___ZNj5b'}).text

        dict = {
            'Company': company_name,
            'brand_name': brand_name,
            'key ingredients': key_ingredients,
            'key benefits': key_benefits,
            'rating': rating_star,
            'number of rating': rating
        }
        details_list.append(dict)
        cnt += 1
        print(cnt)
        df = pd.DataFrame(details_list)
        df.to_csv('medicine_details_new.csv', index=False)
    except Exception as e:
        print(f"Error processing URL: {url}. Error: {e}")

# Close the browser after scraping all URLs
driver.quit()



It helps improve mood and energy levels
Key Ingredients: Berberis Aquifolium
1
Carduus marianus
Error processing URL: https://www.1mg.com/otc/medisynth-jondila-forte-syrup-otc787791. Error: 'NoneType' object has no attribute 'strip'

Key Ingredients: None
2
It helps manage dull, dry and coarse hair
Key Ingredients: Arnica, Jaborandi, Thuja, and Calendula
3
Carduus marianus
Error processing URL: https://www.1mg.com/otc/medisynth-jondila-forte-syrup-otc787798. Error: 'NoneType' object has no attribute 'strip'
Arnica
Key Ingredients: 
4

Key Ingredients: None
5
Carduus marianus
Error processing URL: https://www.1mg.com/otc/medisynth-jondila-forte-syrup-otc787795. Error: 'NoneType' object has no attribute 'strip'
Peppermint, Kapoor, Gandhapuro tail, Nilgiri Tail, Chah tail, Ganjani tail
Key Ingredients: 
6
Plantago Q
Error processing URL: https://www.1mg.com/otc/fourrts-gum-forte-gel-homoeopathic-toothpaste-otc226835. Error: 'NoneType' object has no attribute 'strip'

Key Ingredients: None

KeyboardInterrupt: 

In [ ]:
for i in range(3,13):
    key_ingredients=soup.find('div',{'class':'ProductDescription__description-content___A_qCZ'}).find_all('ul')[i].text
    print( key_ingredients)

In [ ]:
key_ingredients = None
for strong_tag in k.find_all('strong'):
    if strong_tag.get_text() == 'Key Ingredients:':
        key_ingredients = strong_tag.find_next_sibling(text-True).strip()
        break

print("Key Ingredients:", key_ingrediegts)